<a href="https://colab.research.google.com/github/PavanDaniele/drone-person-detection/blob/main/dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up: mount drive + import libraries

In [15]:
# Run this Every time you start a new session
from google.colab import drive
drive.mount('/content/drive') # to mount google drive (to see/access it)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Run this snippet Just one time, to install packages
!pip install imagehash
!pip install pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.8 MB/s eta 0:00:00


In [27]:
from PIL import Image
import imagehash
import os
from itertools import combinations # to generate all possible combinations of a number of elements from a set

from collections import defaultdict, deque
# defaultdict is a special type of dictionary that automatically creates a default value if you access a nonexistent key
# deque is a list-like structure, but optimized for quick additions and removals at both ends.

from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt

# Dataset Preparation

In this notebook, I'm going to prepare the dataset for fine-tuning multiple deep learning models (e.g. YOLO, EfficientDet, SSD + MobileNetV2).
The steps include similarity check, dataset splitting (train/val/test), optional image resizing, and bounding box adaptation.
The goal is to generate separate, clean and model-ready datasets for each architecture to enable fair training and evaluation.

In [9]:
image_folder_path = "/content/drive/MyDrive/projectUPV/datasets/AERALIS"

There are various metrics to calculate image similarity, such as **SSIM** (Structural Similarity Index), **PSNR** (Peak Signal-to-Noise Ratio), and **Cosine Similarity**. \
In our case, I chose to use **Perceptual Hashing** for an initial check because it is fast, robust, and does not require resizing (which is very important since my AERALIS dataset is composed of images from two different datasets).

This technique reduces the image to a binary signature, and then the Hamming Distance is computed to compare the resulting binary hashes.

In [2]:
def get_image_paths(folder_path): # To estract the images file (.jpg) and ignore the .xml and .csv files
  """
  Args:
    folder_path: path to folder containing images
  Returns:
    list of paths to images
  """
  return [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path)
               if f.lower().endswith(('.jpg'))]

In [3]:
def compute_hash(img_path, method):
  """
  Args:
    img_path: path to image
    method: hash method to use
  Returns:
    hash of image
  """
  img = Image.open(img_path).convert("L")  # Grayscale (because the hash algorithms works best when the image is in black and white)

  if method == 'phash':
    return imagehash.phash(img)
  elif method == 'ahash':
    return imagehash.average_hash(img)
  elif method == 'dhash':
    return imagehash.dhash(img)
  else:
    raise ValueError(f"Hash method not supported: {method}")

In [4]:
def compute_all_hashes(image_paths, methods): # Hash calculation for each images
  """
  Args:
    image_paths: list of paths to images
    methods: list of hash methods to use
  Returns:
    dictionary of hashes
  """
  hashes = {method: {} for method in methods} # to create a dictionary and for each method creates an empty sub-dictionary

  for method in methods:
    print(f"\nCalculation {method} for all images")

    for path in image_paths: # cycles over each image path in the image_paths list
      try:
        h = compute_hash(path, method)
        hashes[method][path] = h # saves the calculated hash in the dictionary structure
      except Exception as e:
        print(f"Error with {path}: {e}")

  return hashes

In [5]:
def compare_hashes(hashes, threshold): # Comparison of images in pairs
  """
  Args:
    hashes: dictionary of hashes
    threshold: distance threshold to consider images as similar
  """
  similar_images = []

  for method in hashes:
    print(f"\nRisultats with {method.upper()}:") # .upper() is used to convert the characters to 'uppercase'
    pairs = combinations(hashes[method].items(), 2) # combinations() is used to generate all the possible pairs without repetitions

    for (path1, hash1), (path2, hash2) in pairs:
      dist = hash1 - hash2
      if dist <= threshold:
        similar_images.append({
          'method': method,
          'image1': os.path.basename(path1),
          'image2': os.path.basename(path2),
          'distance': dist
        })
  return similar_images

The Hamming distance between the hashes of two images tells us how visually similar they are.
The result depends on the threshold:

- 1-2 (Very strict) → Only nearly identical images are detected
- 3-5 (Good compromise) → Balances well between false positives and false negatives
- 6-10 (More permissive) → More images are considered similar, but false positives increas

Let's calculate one Hash at time:

In [16]:
HASH_METHODS = ['phash']
HAMMING_THRESHOLD = 5

image_paths = get_image_paths(image_folder_path)
hashes_phash = compute_all_hashes(image_paths, HASH_METHODS)
similar_images_phash = compare_hashes(hashes_phash, HAMMING_THRESHOLD)

# to see how many distine images are considered similar:
img_set = set()
for entry in similar_images_phash:
    img_set.add(entry['image1'])
    img_set.add(entry['image2'])

print(f"Method: {HASH_METHODS} \n")
print(f"Number of similar distinct images: {len(img_set)}")
print(f"Number of All images: {len(image_paths)}")


Calculation phash for all images

Risultats with PHASH:
Method: ['phash'] 

Number of similar distinct images: 1176
Number of All images: 3426


In [17]:
HASH_METHODS = ['ahash']
HAMMING_THRESHOLD = 5

hashes_ahash = compute_all_hashes(image_paths, HASH_METHODS)
similar_images_ahash = compare_hashes(hashes_ahash, HAMMING_THRESHOLD)

img_set = set()
for entry in similar_images_ahash:
    img_set.add(entry['image1'])
    img_set.add(entry['image2'])

print(f"Method: {HASH_METHODS} \n")
print(f"Number of similar distinct images: {len(img_set)}")
print(f"Number of All images: {len(image_paths)}")


Calculation ahash for all images

Risultats with AHASH:
Method: ['ahash'] 

Number of similar distinct images: 2096
Number of All images: 3426


In [18]:
HASH_METHODS = ['dhash']
HAMMING_THRESHOLD = 5

hashes_dhash = compute_all_hashes(image_paths, HASH_METHODS)
similar_images_dhash = compare_hashes(hashes_dhash, HAMMING_THRESHOLD)

img_set = set()
for entry in similar_images_dhash:
    img_set.add(entry['image1'])
    img_set.add(entry['image2'])

print(f"Method: {HASH_METHODS} \n")
print(f"Number of similar distinct images: {len(img_set)}")
print(f"Number of All images: {len(image_paths)}")


Calculation dhash for all images

Risultats with DHASH:
Method: ['dhash'] 

Number of similar distinct images: 1368
Number of All images: 3426


We observed that the number of similar images is quite high.
Instead of simply removing them (which would unnecessarily reduce the dataset size), we adopt a more conservative strategy: we will distribute these similar images carefully across the training, validation, and test sets, in order to prevent potential overfitting or data leakage.

We chose to use only perceptual hashing (pHash), as it is more robust to minor variations in images and less prone to false positives compared to other variants like aHash and dHash.

But now I want to formulate a hypothesis:
Are we sure that all the 1176 images identified as "similar" by pHash are truly similar to each other? \
It could be that these images do not all resemble each other directly, but instead form subgroups (clusters) of mutually similar images, while being different from those in other groups. \
Let’s try to test this assumption.

To model this relationship, I built a data structure based on an undirected graph, where:

 - each node represents an image
 - an edge connects two images if they are considered similar

We then extracted the connected components from this graph, which effectively represent the actual clusters of similar images. These groups will be used to perform a controlled split of the dataset.

In [20]:
# Constructs an oriented graph in which each image is a node and each similar pair is an arc:
def build_similarity_graph(similar_images):
  """
  Args:
    similar_images: list of similar images
  Returns:
    graph: dictionary of graph
  """
  graph = defaultdict(set) # creates a dictionary (key: name_img, val: set_of_images)

  for pair in similar_images: # scrolls each element(=list of dictionaries) of similar_images
    img1 = pair['image1']
    img2 = pair['image2']
    graph[img1].add(img2) # builds the connection in both directions (undirected arc)
    graph[img2].add(img1)

  return graph

In [21]:
# finds the groups (connected components) in the graph:
def find_connected_components(graph):
  """
  Args:
    graph: dictionary of graph
  Returns:
    groups: list of groups
  """
  visited = set() # keeps track of images already visited
  groups = [] # will contain the final groups

  for node in graph: # scrolls each node(=image) in the graph
    if node not in visited: # If the image has not yet been visited, then start a new group
      group = []

      # Start a BFS (Breadth-First Search) with a queue
      # adds the initial node to the queue and marks it as visited
      queue = deque([node])
      visited.add(node)

      while queue: # as long as there are nodes in the tail
        current = queue.popleft() # removes the knot from the head and adds it to the group
        group.append(current)

        # for each neighbor (similar image), if not already visited
        for neighbor in graph[current]:
          if neighbor not in visited:
            visited.add(neighbor) # marks it as visited
            queue.append(neighbor) # puts it in the queue for trial

      # Once the queue is exhausted, the group is complete and it is added to the groups
      groups.append(group)
  return groups

In [24]:
graph = build_similarity_graph(similar_images_phash)
groups = find_connected_components(graph)

print(f"\nFound {len(groups)} groups of similar images.\n")


Found 347 groups of similar images.



Perfect! We were right! \
Now let's do a brief analysis

In [26]:
group_sizes = [len(group) for group in groups]

size_counts = Counter(group_sizes) # count how many groups have size X

# Sort and save to a DataFrame by display
group_distribution = pd.DataFrame(sorted(size_counts.items()), columns=["Group Size", "Number of Groups"])
display(group_distribution)

# Other useful statistics
total_similar_images = sum(group_sizes)
largest_group = max(group_sizes)
average_group_size = total_similar_images / len(groups)

print(f"Total grups: {len(groups)}")
print(f"Total similar images: {total_similar_images}")
print(f"Average group size: {average_group_size:.2f}")
print(f"Largest group: {largest_group} images")

,Group Size,Number of Groups
0,2,221
1,3,49
2,4,24
3,5,18
4,6,10
5,7,5
6,8,5
7,9,2
8,10,2
9,11,1


Total grups: 347
Total similar images: 1176
Average group size: 3.39
Largest group: 45 images


from matplotlib import pyplot as plt
group_distribution['Group Size'].plot(kind='hist', bins=20, title='Group Size')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
group_distribution['Number of Groups'].plot(kind='hist', bins=20, title='Number of Groups')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
group_distribution.plot(kind='scatter', x='Group Size', y='Number of Groups', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Group Size']
  ys = series['Number of Groups']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = group_distribution.sort_values('Group Size', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Group Size')
_ = plt.ylabel('Number of Groups')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Group Size']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Group Size'}, axis=1)
              .sort_values('Group Size', ascending=True))
  xs = counted['Group Size']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = group_distribution.sort_values('Group Size', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Group Size')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
group_distribution['Group Size'].plot(kind='line', figsize=(8, 4), title='Group Size')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
group_distribution['Number of Groups'].plot(kind='line', figsize=(8, 4), title='Number of Groups')
plt.gca().spines[['top', 'right']].set_visible(False)

In [29]:
percentage = total_similar_images / len(image_paths) * 100
print(f"Percentuale di immagini simili nel dataset: {percentage:.2f}%")

Percentuale di immagini simili nel dataset: 34.33%


As we can see, 34.33% of all images in our AERALIS dataset are similar. This is not ideal. \
But don't worry! We can keep all the images and still avoid overfitting or data leakage by using another technique: *Group-Aware Splitting*.

This method is similar to the more classical Stratified Sampling, but it is more suitable for our case. \
So let’s start using this technique to properly create the Training, Validation, and Test sets. \

But before that, I think it could be interesting to see how the pHash results change when we adjust the similarity threshold.

In [30]:
def filter_similar_images(hashes, threshold): # filters similar images by threshold
  """
  Args:
    hashes: dictionary of hashes
    threshold: distance threshold to consider images as similar
  Returns:
    similar_images: list of similar images
  """
  similar_images = []
  pairs = combinations(hashes.items(), 2)

  for (path1, hash1), (path2, hash2) in pairs:
    dist = hash1 - hash2
    if dist <= threshold:
      similar_images.append({'image1': path1, 'image2': path2, 'distance': dist})

  return similar_images

In [32]:
# Let's see Just the pHash case
HASH_METHODS = ['phash']
thresholds_to_try = [3, 5, 7, 10]

hashes_phash_all = compute_all_hashes(image_paths, HASH_METHODS)['phash'] # Extracts only 'phash' from the returned dictionary

results = []
for thresh in thresholds_to_try: # analyzes each threshold
  similar_images = filter_similar_images(hashes_phash_all, thresh) # for each threshold, calculate similar images with that threshold

  # Extracts all the images that appear at least once as similar (without duplicates):
  img_set = set()
  for pair in similar_images:
    img_set.add(pair['image1'])
    img_set.add(pair['image2'])

  results.append({
    "Threshold": thresh,
    "Num Similar Pairs": len(similar_images),
    "Num Similar Distinct Images": len(img_set),
    "Total Images": len(image_paths),
    "Percent Similar (%)": round(len(img_set) / len(image_paths) * 100, 2)
  })


# To see the results let's converts the list of results to a DataFrame pandas
df_results = pd.DataFrame(results)
display(df_results)


Calculation phash for all images


,Threshold,Num Similar Pairs,Num Similar Distinct Images,Total Images,Percent Similar (%)
0,3,1450,839,3426,24.49
1,5,2307,1176,3426,34.33
2,7,3161,1494,3426,43.61
3,10,4960,1971,3426,57.53


from matplotlib import pyplot as plt
df_results['Threshold'].plot(kind='hist', bins=20, title='Threshold')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_results['Num Similar Pairs'].plot(kind='hist', bins=20, title='Num Similar Pairs')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df_results['Num Similar Distinct Images'].plot(kind='hist', bins=20, title='Num Similar Distinct Images')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Total Images']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Total Images'}, axis=1)
              .sort_values('Total Images', ascending=True))
  xs = counted['Total Images']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = df_results.sort_values('Total Images', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Total Images')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
df_results['Threshold'].plot(kind='line', figsize=(8, 4), title='Threshold')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Num Similar Pairs'].plot(kind='line', figsize=(8, 4), title='Num Similar Pairs')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Num Similar Distinct Images'].plot(kind='line', figsize=(8, 4), title='Num Similar Distinct Images')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df_results['Percent Similar (%)'].plot(kind='line', figsize=(8, 4), title='Percent Similar (%)')
plt.gca().spines[['top', 'right']].set_visible(False)

We observe that as the threshold increases, the number of pairs considered similar also grows, and consequently, so does the percentage of images involved.

Observations:
- At lower thresholds (e.g., 3), only strongly similar images are identified, but many less obvious duplicates may be missed.

- At higher thresholds (e.g., 10), there's a risk of including different images that only share generic visual elements (false positives).

- Threshold 5 proves to be a good compromise, balancing precision and coverage.